### Libraries and Datasets

In [1]:
import os
import py7zr
import pandas as pd

In [2]:
if not os.path.exists('../Extracted_Dataset'):
    paths = []
    for dirname, _, filenames in os.walk('../favorita-grocery-sales-forecasting'):
        for filename in filenames:
            print(os.path.join(dirname, filename))
            paths.append(os.path.join(dirname, filename))

In [3]:
if not os.path.exists('../Extracted_Dataset'):
    for filenames in paths:
        with py7zr.SevenZipFile(filenames, mode='r') as z_ref:
            z_ref.extractall(path='../Extracted_Dataset')

### Train and Test Data

In [4]:
train_data = pd.read_csv('../Extracted_Dataset/train.csv')
test_data = pd.read_csv('../Extracted_Dataset/test.csv')
train_data['date'] = pd.to_datetime(train_data['date'])
test_data['date'] = pd.to_datetime(test_data['date'])

C:\Users\Tharusha\AppData\Local\Temp\ipykernel_11508\289034664.py:1: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  train_data = pd.read_csv('../Extracted_Dataset/train.csv')


In [5]:
train_data

,id,date,store_nbr,item_nbr,unit_sales,onpromotion
0,0,2013-01-01,25,103665,7.0,NaN
1,1,2013-01-01,25,105574,1.0,NaN
2,2,2013-01-01,25,105575,2.0,NaN
3,3,2013-01-01,25,108079,1.0,NaN
4,4,2013-01-01,25,108701,1.0,NaN
...,...,...,...,...,...,...
125497035,125497035,2017-08-15,54,2089339,4.0,False
125497036,125497036,2017-08-15,54,2106464,1.0,True
125497037,125497037,2017-08-15,54,2110456,192.0,False
125497038,125497038,2017-08-15,54,2113914,198.0,True


In [6]:
test_data

,id,date,store_nbr,item_nbr,onpromotion
0,125497040,2017-08-16,1,96995,False
1,125497041,2017-08-16,1,99197,False
2,125497042,2017-08-16,1,103501,False
3,125497043,2017-08-16,1,103520,False
4,125497044,2017-08-16,1,103665,False
...,...,...,...,...,...
3370459,128867499,2017-08-31,54,2132163,False
3370460,128867500,2017-08-31,54,2132318,False
3370461,128867501,2017-08-31,54,2132945,False
3370462,128867502,2017-08-31,54,2132957,False


In [7]:
train_items = set(train_data['item_nbr'].to_list())
test_items = set(test_data['item_nbr'].to_list())

print(f"Number of items in both train and test: {len(train_items.intersection(test_items))}")

Number of items in both train and test: 3841


In [8]:
train_stores = set(train_data['store_nbr'].to_list())
test_stores = set(test_data['store_nbr'].to_list())

print(f"Number of stores in both train and test: {len(train_stores.intersection(test_stores))}")

Number of stores in both train and test: 54


In [9]:
training_data_available_ts = train_data.groupby(['store_nbr', 'item_nbr'])['date'].count().reset_index()
training_data_available_ts.columns = ['store_nbr', 'item_nbr', 'count']
training_data_available_ts.sort_values(by='count', ascending=False, inplace=True)
training_data_available_ts

,store_nbr,item_nbr,count
126176,41,564533,1679
161702,51,305229,1679
161804,51,364606,1679
161575,51,208514,1679
161571,51,208384,1679
...,...,...,...
4681,2,907686,1
94597,30,1696052,1
94469,30,1503891,1
105908,34,1695845,1


In [10]:
training_data_available_ts[training_data_available_ts['count']<20]

,store_nbr,item_nbr,count
125450,40,2037148,19
114605,37,1430082,19
174592,54,2047496,19
129106,41,2113343,19
134361,43,1309672,19
...,...,...,...
4681,2,907686,1
94597,30,1696052,1
94469,30,1503891,1
105908,34,1695845,1


In [11]:
train_data.isna().sum()

id                    0
date                  0
store_nbr             0
item_nbr              0
unit_sales            0
onpromotion    21657651
dtype: int64

In [12]:
test_data.isna().sum()

id             0
date           0
store_nbr      0
item_nbr       0
onpromotion    0
dtype: int64

### Store Information

In [13]:
store_data = pd.read_csv('../Extracted_Dataset/stores.csv')
store_data.head()

,store_nbr,city,state,type,cluster
0,1,Quito,Pichincha,D,13
1,2,Quito,Pichincha,D,13
2,3,Quito,Pichincha,D,8
3,4,Quito,Pichincha,D,9
4,5,Santo Domingo,Santo Domingo de los Tsachilas,D,4


In [14]:
store_data.isna().sum()

store_nbr    0
city         0
state        0
type         0
cluster      0
dtype: int64

In [15]:
store_data['type'].value_counts()

type
D    18
C    15
A     9
B     8
E     4
Name: count, dtype: int64

In [16]:
store_data['cluster'].value_counts()

cluster
3     7
10    6
6     6
15    5
13    4
14    4
11    3
4     3
8     3
1     3
2     2
9     2
7     2
12    1
16    1
5     1
17    1
Name: count, dtype: int64

### Item Data

In [17]:
item_data = pd.read_csv('../Extracted_Dataset/items.csv')
item_data

,item_nbr,family,class,perishable
0,96995,GROCERY I,1093,0
1,99197,GROCERY I,1067,0
2,103501,CLEANING,3008,0
3,103520,GROCERY I,1028,0
4,103665,BREAD/BAKERY,2712,1
...,...,...,...,...
4095,2132318,GROCERY I,1002,0
4096,2132945,GROCERY I,1026,0
4097,2132957,GROCERY I,1068,0
4098,2134058,BEVERAGES,1124,0


In [18]:
item_data.isna().sum()

item_nbr      0
family        0
class         0
perishable    0
dtype: int64

In [19]:
item_data['family'].value_counts()

family
GROCERY I                     1334
BEVERAGES                      613
CLEANING                       446
PRODUCE                        306
DAIRY                          242
PERSONAL CARE                  153
BREAD/BAKERY                   134
HOME CARE                      108
DELI                            91
MEATS                           84
HOME AND KITCHEN I              77
LIQUOR,WINE,BEER                73
FROZEN FOODS                    55
POULTRY                         54
HOME AND KITCHEN II             45
EGGS                            41
CELEBRATION                     31
LAWN AND GARDEN                 26
PREPARED FOODS                  26
LADIESWEAR                      21
LINGERIE                        20
AUTOMOTIVE                      20
BEAUTY                          19
PLAYERS AND ELECTRONICS         17
SCHOOL AND OFFICE SUPPLIES      15
GROCERY II                      14
PET SUPPLIES                    14
SEAFOOD                          8
MAGAZINES    

In [20]:
item_data['family'].nunique()

33

In [21]:
item_data['class'].nunique()

337

### Transaction Data

In [22]:
transcation_data = pd.read_csv('../Extracted_Dataset/transactions.csv')
transcation_data['date'] = pd.to_datetime(transcation_data['date'])
transcation_data

,date,store_nbr,transactions
0,2013-01-01,25,770
1,2013-01-02,1,2111
2,2013-01-02,2,2358
3,2013-01-02,3,3487
4,2013-01-02,4,1922
...,...,...,...
83483,2017-08-15,50,2804
83484,2017-08-15,51,1573
83485,2017-08-15,52,2255
83486,2017-08-15,53,932


In [23]:
transcation_data.isna().sum()

date            0
store_nbr       0
transactions    0
dtype: int64

### Holiday Data

In [24]:
holiday_data = pd.read_csv('../Extracted_Dataset/holidays_events.csv')
holiday_data['date'] = pd.to_datetime(holiday_data['date'])
holiday_data

,date,type,locale,locale_name,description,transferred
0,2012-03-02,Holiday,Local,Manta,Fundacion de Manta,False
1,2012-04-01,Holiday,Regional,Cotopaxi,Provincializacion de Cotopaxi,False
2,2012-04-12,Holiday,Local,Cuenca,Fundacion de Cuenca,False
3,2012-04-14,Holiday,Local,Libertad,Cantonizacion de Libertad,False
4,2012-04-21,Holiday,Local,Riobamba,Cantonizacion de Riobamba,False
...,...,...,...,...,...,...
345,2017-12-22,Additional,National,Ecuador,Navidad-3,False
346,2017-12-23,Additional,National,Ecuador,Navidad-2,False
347,2017-12-24,Additional,National,Ecuador,Navidad-1,False
348,2017-12-25,Holiday,National,Ecuador,Navidad,False


In [25]:
holiday_data.isna().sum()

date           0
type           0
locale         0
locale_name    0
description    0
transferred    0
dtype: int64

In [26]:
holiday_data['type'].value_counts()

type
Holiday       221
Event          56
Additional     51
Transfer       12
Bridge          5
Work Day        5
Name: count, dtype: int64

In [27]:
holiday_data.loc[(holiday_data['date'] >= '2016-04-17') & (holiday_data['date'] <= '2016-04-25')]

,date,type,locale,locale_name,description,transferred
220,2016-04-17,Event,National,Ecuador,Terremoto Manabi+1,False
221,2016-04-18,Event,National,Ecuador,Terremoto Manabi+2,False
222,2016-04-19,Event,National,Ecuador,Terremoto Manabi+3,False
223,2016-04-20,Event,National,Ecuador,Terremoto Manabi+4,False
224,2016-04-21,Holiday,Local,Riobamba,Cantonizacion de Riobamba,False
225,2016-04-21,Event,National,Ecuador,Terremoto Manabi+5,False
226,2016-04-22,Event,National,Ecuador,Terremoto Manabi+6,False
227,2016-04-23,Event,National,Ecuador,Terremoto Manabi+7,False
228,2016-04-24,Event,National,Ecuador,Terremoto Manabi+8,False
229,2016-04-25,Event,National,Ecuador,Terremoto Manabi+9,False


### Oil Data

In [28]:
oil_data = pd.read_csv('../Extracted_Dataset/oil.csv')
oil_data['date'] = pd.to_datetime(oil_data['date'])
oil_data

,date,dcoilwtico
0,2013-01-01,NaN
1,2013-01-02,93.14
2,2013-01-03,92.97
3,2013-01-04,93.12
4,2013-01-07,93.20
...,...,...
1213,2017-08-25,47.65
1214,2017-08-28,46.40
1215,2017-08-29,46.46
1216,2017-08-30,45.96


In [29]:
oil_data.isna().sum()

date           0
dcoilwtico    43
dtype: int64

### Dataset Preparation

In [30]:
train_data = train_data.merge(store_data, on='store_nbr', how='left')
train_data = train_data.merge(item_data, on='item_nbr', how='left')
train_data

,id,date,store_nbr,item_nbr,unit_sales,onpromotion,city,state,type,cluster,family,class,perishable
0,0,2013-01-01,25,103665,7.0,NaN,Salinas,Santa Elena,D,1,BREAD/BAKERY,2712,1
1,1,2013-01-01,25,105574,1.0,NaN,Salinas,Santa Elena,D,1,GROCERY I,1045,0
2,2,2013-01-01,25,105575,2.0,NaN,Salinas,Santa Elena,D,1,GROCERY I,1045,0
3,3,2013-01-01,25,108079,1.0,NaN,Salinas,Santa Elena,D,1,GROCERY I,1030,0
4,4,2013-01-01,25,108701,1.0,NaN,Salinas,Santa Elena,D,1,DELI,2644,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
125497035,125497035,2017-08-15,54,2089339,4.0,False,El Carmen,Manabi,C,3,GROCERY I,1006,0
125497036,125497036,2017-08-15,54,2106464,1.0,True,El Carmen,Manabi,C,3,BEVERAGES,1148,0
125497037,125497037,2017-08-15,54,2110456,192.0,False,El Carmen,Manabi,C,3,BEVERAGES,1120,0
125497038,125497038,2017-08-15,54,2113914,198.0,True,El Carmen,Manabi,C,3,CLEANING,3040,0


In [31]:
average_sales = train_data.groupby("item_nbr")["unit_sales"].mean().reset_index()
average_sales.columns = ["item_nbr", "avg_unit_sales"]

In [32]:
split_datasets = {f"cluster_{c}_family_{f}": group for (c, f), group in train_data.groupby(["cluster", "family"])}

In [33]:
# This dataset will be used for the model training
chosen_df = split_datasets['cluster_1_family_AUTOMOTIVE']
chosen_df = chosen_df.merge(average_sales, on="item_nbr", how="left")

In [34]:
chosen_df

,id,date,store_nbr,item_nbr,unit_sales,onpromotion,city,state,type,cluster,family,class,perishable,avg_unit_sales
0,18825,2013-01-02,24,121964,1.0,NaN,Guayaquil,Guayas,D,1,AUTOMOTIVE,6810,0,1.473475
1,21426,2013-01-02,27,121964,1.0,NaN,Daule,Guayas,D,1,AUTOMOTIVE,6810,0,1.473475
2,21565,2013-01-02,27,270522,2.0,NaN,Daule,Guayas,D,1,AUTOMOTIVE,6806,0,2.168911
3,21589,2013-01-02,27,312317,1.0,NaN,Daule,Guayas,D,1,AUTOMOTIVE,6810,0,1.511356
4,21731,2013-01-02,27,461432,1.0,NaN,Daule,Guayas,D,1,AUTOMOTIVE,6810,0,1.627619
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11816,125439980,2017-08-15,25,1969477,1.0,False,Salinas,Santa Elena,D,1,AUTOMOTIVE,6848,0,1.277936
11817,125439981,2017-08-15,25,1969863,2.0,False,Salinas,Santa Elena,D,1,AUTOMOTIVE,6848,0,1.737062
11818,125441739,2017-08-15,27,511394,1.0,False,Daule,Guayas,D,1,AUTOMOTIVE,6810,0,1.305147
11819,125442332,2017-08-15,27,1147731,1.0,False,Daule,Guayas,D,1,AUTOMOTIVE,6806,0,1.758419


In [35]:
#Missing values filled with the mode for the time being
mode_value = chosen_df["onpromotion"].mode()[0] if not chosen_df["onpromotion"].mode().empty else False
chosen_df["onpromotion"].fillna(mode_value, inplace=True)

C:\Users\Tharusha\AppData\Local\Temp\ipykernel_11508\3609755632.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  chosen_df["onpromotion"].fillna(mode_value, inplace=True)
C:\Users\Tharusha\AppData\Local\Temp\ipykernel_11508\3609755632.py:3: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  chosen_df["onpromotion

In [36]:
chosen_df

,id,date,store_nbr,item_nbr,unit_sales,onpromotion,city,state,type,cluster,family,class,perishable,avg_unit_sales
0,18825,2013-01-02,24,121964,1.0,False,Guayaquil,Guayas,D,1,AUTOMOTIVE,6810,0,1.473475
1,21426,2013-01-02,27,121964,1.0,False,Daule,Guayas,D,1,AUTOMOTIVE,6810,0,1.473475
2,21565,2013-01-02,27,270522,2.0,False,Daule,Guayas,D,1,AUTOMOTIVE,6806,0,2.168911
3,21589,2013-01-02,27,312317,1.0,False,Daule,Guayas,D,1,AUTOMOTIVE,6810,0,1.511356
4,21731,2013-01-02,27,461432,1.0,False,Daule,Guayas,D,1,AUTOMOTIVE,6810,0,1.627619
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11816,125439980,2017-08-15,25,1969477,1.0,False,Salinas,Santa Elena,D,1,AUTOMOTIVE,6848,0,1.277936
11817,125439981,2017-08-15,25,1969863,2.0,False,Salinas,Santa Elena,D,1,AUTOMOTIVE,6848,0,1.737062
11818,125441739,2017-08-15,27,511394,1.0,False,Daule,Guayas,D,1,AUTOMOTIVE,6810,0,1.305147
11819,125442332,2017-08-15,27,1147731,1.0,False,Daule,Guayas,D,1,AUTOMOTIVE,6806,0,1.758419


In [37]:
chosen_df.to_csv('cluster_1_family_AUTOMOTIVE.csv', index=False)